<a href="https://colab.research.google.com/github/YeaAyuni/whs-runner/blob/main/Whs_Runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

In [ ]:
#@title Packages
!sudo apt update && sudo apt upgrade
!sudo apt install ffmpeg

!pip install gradio
!pip install git+https://github.com/openai/whisper.git
!pip install yt_dlp

In [ ]:
#@title Modules
import os
from pathlib import Path
import shutil
from IPython.display import clear_output

os.makedirs("/content/whisper/output", exist_ok=True)
os.makedirs("/content/whisper/input", exist_ok=True)

# Run

In [ ]:
#@title Download Audio dari Youtube
audioURL = "https://www.youtube.com/watch?v=L6SNXnEErH0" #@param {type:"string"}
fileName = "linting-daun" #@param {type:"string"}
#@markdown ---
import yt_dlp
import os

os.chdir('/content/whisper/input')

def str_escape_space(text: str):
    return '_'.join(text.split())

def getAudioFromUrl(songTitle, songUrl, output_format='wav'):
    song_title = songTitle
    song_title = str_escape_space(song_title)
    url = songUrl

    ydl_options = {
        'format': 'bestaudio/best',
        'postprocessors': [
            {
                'key': 'FFmpegExtractAudio',
                'preferredcodec': output_format
            }
        ],
        'outtmpl': f'{song_title}'
    }

    try:
        ydl = yt_dlp.YoutubeDL(ydl_options)
        ydl.download([url])
        print('OK')
    except Exception as e:
        print(e)
    return fileName, output_format

out = getAudioFromUrl(songTitle=fileName, songUrl=audioURL)
file_from_youtube_path = f"/content/whisper/input/{out[0]}.{out[1]}"
print(f"Audio Location Path: {file_from_youtube_path}")



In [ ]:
#@title Generate

audio_path = "/content/whisper/input/linting-daun.wav" #@param {type:"string"}
task = "transcribe" #@param ["transcribe","translate"]
source_language = "id" #@param ["auto","en", "ja", "id", "ko"]
model_type = "large" #@param ["tiny", "base", "small", "medium", "large"]
output_format = "srt" #@param ["all", "srt", "json", "txt", "vtt", "tsv"]

output_dir = os.path.join("/content/whisper/output", Path(audio_path).stem)

if os.path.isdir(output_dir):
  shutil.rmtree(output_dir)

os.makedirs(output_dir)
os.chdir(output_dir)

print("===========================================")
print(f"Output Location : {output_dir}")
print("Generating...")
if source_language == "auto":
  !whisper "$audio_path" --model "$model_type" --task "$task" --output_format "$output_format"
else:
  !whisper "$audio_path" --language "$source_language" --model "$model_type" --task "$task" --output_format "$output_format"


In [ ]:
#@title Download Output
from google.colab import files

output_folder = "" #@param {type:'string'}
#@markdown if empty will use <code>output_dir</code> from **Generate** cell.
folder = output_dir if output_folder == "" else output_folder


target_folder = Path(folder).stem
output_zip = f"{folder}/{Path(folder).stem}-outputs.zip"
# !zip -r "$output_zip" "$folder"
os.chdir(Path(folder).parent)

if os.path.exists(output_zip):
  os.remove(output_zip)

!zip -r "$output_zip" "$target_folder"
# files.download(output_zip)

# Web UI

In [ ]:
import gradio as gr

def VideoBox():
    with gr.Row():
      urlBox = gr.Textbox(placeholder="Video url/path", label="Input Video Path")
      previewButton = gr.Button(interactive=True, value="Show Video")

    with gr.Row():
      with gr.Accordion(label="Video Output"):
      # createEmptyContaienr
        videoPlayer = gr.Video(visible=False)
        # functionForEmptyContainer
        def showVideo(urlBox):
          videoBox = gr.Video.update(visible=True, value=urlBox, autoplay=True)
          return videoBox
        # end of

    previewButton.click(fn=showVideo, outputs=videoPlayer , inputs=urlBox)

with gr.Blocks() as app:

  # video configuration
  with gr.Box():
    gr.Markdown(value="Configuration")
    VideoBox()

if __name__ == "__main__":
    app.launch(debug=True, share=True)